<a href="https://colab.research.google.com/github/LokiAndere/MIARFID-RNA-2019-20/blob/master/cifar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization as BN
from keras.layers import GaussianNoise as GN
from keras.optimizers import SGD

from keras.callbacks import LearningRateScheduler as LRS
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator


batch_size = 100
num_classes = 10
epochs = 150


#### LOAD AND TRANSFORM
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print(x_train.shape)
print(x_test.shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


## DEFINE A DATA AUGMENTATION GENERATOR

datagen = ImageDataGenerator(
  featurewise_center=True,
  featurewise_std_normalization=True,
  width_shift_range=0.2,
  height_shift_range=0.2,
  rotation_range=40,
  zoom_range=[1.0,1.2],
  horizontal_flip=True)


###########################################################
# Now this is necessary due to the feature normalization: #
datagen.fit(x_train)

testdatagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
)

testdatagen.fit(x_train)
###########################################################


## DEF A BLOCK CONV + BN + GN + CONV + BN + GN + MAXPOOL 
def CBGN(model,filters,ishape=0):
  if (ishape!=0):
    model.add(Conv2D(filters, (3, 3), padding='same',
                 input_shape=ishape))
  else:
    model.add(Conv2D(filters, (3, 3), padding='same'))

    
  model.add(BN())
  model.add(GN(0.3))
  model.add(Activation('relu'))

  model.add(Conv2D(filters, (3, 3), padding='same'))
  model.add(BN())
  model.add(GN(0.3))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  return model

  
## DEF NN TOPOLOGY  
model = Sequential()

model=CBGN(model,32,x_train.shape[1:])
model=CBGN(model,64)
model=CBGN(model,128)
model=CBGN(model,256)
model=CBGN(model,512)

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))

model.add(Dense(num_classes))
model.add(Activation('softmax'))


model.summary()


## OPTIM AND COMPILE
opt = SGD(lr=0.1, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# DEFINE A LEARNING RATE SCHEDULER
def scheduler(epoch):
    if epoch < 50:
        return .1
    elif epoch < 100:
        return 0.01
    else:
        return 0.001

set_lr = LRS(scheduler)


mcp_save = ModelCheckpoint ('drive/My Drive/sss.hdf5', save_best_only = True, monitor = 'val_acc', mode = 'auto')

## TRAINING with DA and LRA
history=model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                            steps_per_epoch=len(x_train) / batch_size, 
                            epochs=epochs,
                            validation_data=testdatagen.flow(x_test, y_test),
                            callbacks=[set_lr, mcp_save],
                            verbose=1)











(50000, 32, 32, 3)
(10000, 32, 32, 3)
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_21 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
gaussian_noise_21 (GaussianN (None, 32, 32, 32)        0         
_________________________________________________________________
activation_25 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_22 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
gaussian_noise_2

NameError: ignored

In [0]:
import keras
from keras.models import load_model
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator

model = load_model ('drive/My Drive/sss.hdf5')
model.summary()

num_classes = 10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('Train', x_train.shape, y_train.shape)
print('Test', (x_test.shape, y_test.shape))

print('Train', x_train.min(), x_train.max(), x_train.mean(), x_train.std())
print('Test', x_test.min(), x_test.max(), x_test.mean(), x_test.std())

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print(x_train.shape)
print(x_test.shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

testdatagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
)

testdatagen.fit(x_test)

iterator = testdatagen.flow(x_test, y_test)

for i in range (2):
  score = model.evaluate(iterator, verbose=i)
  print('Test loss:', score[0])
  print('Test accuracy:', score[1])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 32, 32, 32)        0         
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
gaussian_noise_2 (GaussianNo (None, 32, 32, 32)       

In [0]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization as BN
from keras.layers import GaussianNoise as GN
from keras.optimizers import SGD

from keras.callbacks import LearningRateScheduler as LRS
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator


batch_size = 50
num_classes = 10
epochs = 150

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print(x_train.shape)
print(x_test.shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


## DEFINE A DATA AUGMENTATION GENERATOR

datagen = ImageDataGenerator(
  featurewise_center=True,
  featurewise_std_normalization=True,
  width_shift_range=0.2,
  height_shift_range=0.2,
  rotation_range=20,
  shear_range=0.2,
  zoom_range=[1.0,1.2],
  horizontal_flip=True)


###########################################################
# Now this is necessary due to the feature normalization: #
datagen.fit(x_train)

testdatagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
)

testdatagen.fit(x_train)
###########################################################


## DEF A BLOCK CONV + BN + GN + CONV + BN + GN + MAXPOOL 
def CBGN(model,filters,ishape=0):
  if (ishape!=0):
    model.add(Conv2D(filters, kernel_size=(3, 3),padding='same',
                 input_shape=ishape))
  else:
    model.add(Conv2D(filters, kernel_size=(3, 3),padding='same'))

  model.add(BN())
  model.add(GN(0.3))
  model.add(Activation('relu'))
  model.add(Conv2D(filters, (3, 3), padding='same'))
  model.add(BN())
  model.add(GN(0.3))
  model.add(Activation('relu'))
  model.add(Conv2D(filters, (5, 5), padding='same'))
  model.add(BN())
  model.add(GN(0.3))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  return model

  
## DEF NN TOPOLOGY  
model = Sequential()

model=CBGN(model,32,x_train.shape[1:])
model=CBGN(model,64)
model=CBGN(model,128)

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(BN())
model.add(Dropout(0.4))

model.add(Dense(num_classes))
model.add(Activation('softmax'))


model.summary()


## OPTIM AND COMPILE
opt = SGD(lr=0.1, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# DEFINE A LEARNING RATE SCHEDULER
def scheduler(epoch):
    if epoch < 80:
        return 1e-3
    elif epoch < 120:
        return 1e-4
    elif epoch < 150:
        return 1e-5
    elif epoch < 180:
        return 1e-6    
    else:
        return 5e-7

set_lr = LRS(scheduler)


mcp_save = ModelCheckpoint ('drive/My Drive/sss2.hdf5', save_best_only = True, monitor = 'val_acc', mode = 'auto')

## TRAINING with DA and LRA
history=model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                            steps_per_epoch=len(x_train) / batch_size, 
                            epochs=epochs,
                            validation_data=testdatagen.flow(x_test, y_test),
                            callbacks=[set_lr, mcp_save],
                            verbose=1)


scores = model.evaluate(datagen.flow(x_test, y_test), verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

(50000, 32, 32, 3)
(10000, 32, 32, 3)
Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_86 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_90 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
gaussian_noise_86 (GaussianN (None, 32, 32, 32)        0         
_________________________________________________________________
activation_86 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_87 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_91 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
gaussian_noise_

In [0]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization as BN
from keras.layers import GaussianNoise as GN
from keras.optimizers import SGD
import numpy as np
from keras.callbacks import LearningRateScheduler as LRS
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2


batch_size = 75
num_classes = 10
epochs = 150

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

print(x_train.shape)
print(x_test.shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

datagen = ImageDataGenerator(
  width_shift_range=0.2,
  height_shift_range=0.2,
  rotation_range=20,
  shear_range=0.2,
  zoom_range=[1.0,1.2],
  horizontal_flip=True)

def CCP(model,filters,ishape=0):
  if (ishape!=0):
    model.add(ZeroPadding2D(((1,1),(1,1)), input_shape=ishape))
  else:
    model.add(ZeroPadding2D(((1,1),(1,1))))

  model.add(Activation('relu'))
  model.add(Conv2D(filters, (3, 3),kernel_regularizer=l2(0.01)))
  model.add(Activation('relu'))
  model.add(ZeroPadding2D(((1,1),(1,1))))
  model.add(Conv2D(filters, (3, 3),kernel_regularizer=l2(0.01)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D((2,2), strides=(2,2)))
  
  return model

def DR(model, neurons):

  model.add(Dense(neurons))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  
  return model
  
## DEF NN TOPOLOGY  
model = Sequential()

model=CCP(model,64,x_train.shape[1:])
model=CCP(model,128)
model=CCP(model,512)
model.add(Flatten())
model=DR(model,1024)
model=DR(model,2048)
model.add(Dense(num_classes))
model.add(Activation('softmax'))


model.summary()

opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt_rms,
              metrics=['accuracy'])

def scheduler(epoch):
    if epoch < 80:
        return 1e-3
    elif epoch < 120:
        return 1e-4
    elif epoch < 150:
        return 1e-5
    elif epoch < 180:
        return 1e-6    
    else:
        return 5e-7

set_lr = LRS(scheduler)


mcp_save = ModelCheckpoint ('drive/My Drive/sssn.hdf5', save_best_only = True, monitor = 'val_acc', mode = 'auto')

history=model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                            steps_per_epoch=len(x_train) / batch_size, 
                            epochs=epochs,
                            validation_data=(x_test, y_test),
                            callbacks=[set_lr, mcp_save],
                            verbose=1)

scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print(f'Test loss {scores[0]}\nTest accuracy {scores[1]}')

Using TensorFlow backend.


170500096/170498071 [==============================] - 2s 0us/step
(50000, 32, 32, 3)
(10000, 32, 32, 3)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 34, 34, 3)         0         
_________________________________________________________________
activation_1 (Activation)    (None, 34, 34, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 34, 34, 64)        0         
___________________________

KeyboardInterrupt: ignored

In [0]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization as BN
from keras.layers import GaussianNoise as GN
from keras.optimizers import SGD
import numpy as np
from keras.callbacks import LearningRateScheduler as LRS
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2


batch_size = 50
num_classes = 10
epochs = 150

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

print(x_train.shape)
print(x_test.shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


datagen = ImageDataGenerator(
  width_shift_range=0.2,
  height_shift_range=0.2,
  rotation_range=20,
  shear_range=0.2,
  zoom_range=[1.0,1.2],
  horizontal_flip=True)

def VGG(model,filters,ishape=0):
  if (ishape!=0):
    model.add(Conv2D(filters, (3, 3), padding='same', input_shape=ishape))
  else:
    model.add(Conv2D(filters, (3, 3), padding='same'))

  model.add(BN())
  model.add(Activation('relu'))
  model.add(MaxPooling2D((2,2), strides=(2,2)))
  
  return model

def DR(model, neurons):

  model.add(Dense(neurons))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  
  return model

model = Sequential()

model=VGG(model,64,x_train.shape[1:])
model=VGG(model,128)
model=VGG(model,256)
model=VGG(model,512)
model=VGG(model,512)
model.add(AveragePooling2D((1,1), strides=1))
model.add(Flatten())
model.add(Dense(num_classes))
model.add(Activation('softmax'))


model.summary()

opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt_rms,
              metrics=['accuracy'])

def scheduler(epoch):
    if epoch < 80:
        return 1e-3
    elif epoch < 120:
        return 1e-4
    elif epoch < 150:
        return 1e-5
    elif epoch < 180:
        return 1e-6    
    else:
        return 5e-7

set_lr = LRS(scheduler)


mcp_save = ModelCheckpoint ('drive/My Drive/sss3.hdf5', save_best_only = True, monitor = 'val_acc', mode = 'auto')

history=model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                            steps_per_epoch=len(x_train) / batch_size, 
                            epochs=epochs,
                            validation_data=(x_test, y_test),
                            callbacks=[set_lr, mcp_save],
                            verbose=1)

scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print(f'Test loss {scores[0]}\nTest accuracy {scores[1]}')


(50000, 32, 32, 3)
(10000, 32, 32, 3)


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
activation_13 (Activation)   (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 128)       512       
_________________________________________________________________
activation_14 

In [0]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, DepthwiseConv2D
from keras.layers import MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from keras.layers import ZeroPadding2D
from keras.layers.normalization import BatchNormalization as BN
from keras.layers import GaussianNoise as GN
from keras.optimizers import SGD
import numpy as np
from keras.callbacks import LearningRateScheduler as LRS
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2


batch_size = 50
num_classes = 10
epochs = 100

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

print(x_train.shape)
print(x_test.shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

datagen = ImageDataGenerator(
  width_shift_range=0.2,
  height_shift_range=0.2,
  rotation_range=20,
  shear_range=0.2,
  zoom_range=[1.0,1.2],
  horizontal_flip=True)

def VGG(model,filters,ishape=0):
  if (ishape!=0):
    model.add(DepthwiseConv2D(filters, (3, 3), padding='same', input_shape=ishape))
  else:
    model.add(DepthwiseConv2D(filters, (3, 3), padding='same'))

  model.add(DepthwiseConv2D(filters, (3, 3), padding='same'))
  model.add(BN())
  model.add(Activation('relu'))
  model.add(MaxPooling2D((1,1), strides=(1,1)))
  
  return model

def DR(model, neurons):

  model.add(Dense(neurons))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  
  return model

model = Sequential()

model=VGG(model,32,x_train.shape[1:])
model=VGG(model,64)
model=VGG(model,128)
model.add(AveragePooling2D((1,1), strides=1))
model.add(Flatten())
model=DR(model, 512)
model.add(Dense(num_classes))
model.add(Activation('softmax'))


model.summary()

opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

def scheduler(epoch):
    if epoch < 80:
        return 1e-3
    elif epoch < 120:
        return 1e-4
    elif epoch < 150:
        return 1e-5
    elif epoch < 180:
        return 1e-6    
    else:
        return 5e-7

set_lr = LRS(scheduler)


mcp_save = ModelCheckpoint ('drive/My Drive/sss4.hdf5', save_best_only = True, monitor = 'val_acc', mode = 'auto')

history=model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                            steps_per_epoch=len(x_train) / batch_size, 
                            epochs=epochs,
                            validation_data=(x_test, y_test),
                            callbacks=[set_lr, mcp_save],
                            verbose=1)

scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print(f'Test loss {scores[0]}\nTest accuracy {scores[1]}')


(50000, 32, 32, 3)
(10000, 32, 32, 3)
Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
depthwise_conv2d_45 (Depthwi (None, 11, 11, 3)         3075      
_________________________________________________________________
depthwise_conv2d_46 (Depthwi (None, 4, 4, 3)           3075      
_________________________________________________________________
batch_normalization_28 (Batc (None, 4, 4, 3)           12        
_________________________________________________________________
activation_45 (Activation)   (None, 4, 4, 3)           0         
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 4, 4, 3)           0         
_________________________________________________________________
depthwise_conv2d_47 (Depthwi (None, 2, 2, 3)           12291     
_________________________________________________________________
depthwise_conv2

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, DepthwiseConv2D
from keras.layers import MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from keras.layers import ZeroPadding2D
from keras.layers.normalization import BatchNormalization as BN
from keras.layers import GaussianNoise as GN
from keras.optimizers import SGD
import numpy as np
from keras.callbacks import LearningRateScheduler as LRS
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2


batch_size = 50
num_classes = 10
epochs = 100

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

print(x_train.shape)
print(x_test.shape)
print(y_train[0], y_train[1])
print(y_train.shape)
print(x_train[0].shape)

Using TensorFlow backend.


170500096/170498071 [==============================] - 2s 0us/step
(50000, 32, 32, 3)
(10000, 32, 32, 3)
[6] [9]
(50000, 1)
(32, 32, 3)


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
